In [1]:
import nltk
import string
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from sklearn.manifold import TSNE
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding

Using TensorFlow backend.


In [2]:
data = pd.read_excel("WithCat2.xlsx")
data.index = range(511)
data['Category'].value_counts()

H     104
WP     92
TO     70
TB     62
CB     56
TA     47
CH     35
R      19
EP     16
A       4
CC      4
T       2
Name: Category, dtype: int64

In [3]:
fdata = data.loc[0:511,['Company']]
ldata = data.loc[0:511,['Category']]

In [4]:
print(fdata)
print(ldata)

                                               Company
0                          Barcelona Convention Bureau
1           Belgium - Visit.Brussels Convention Bureau
2       Cologne Convention Bureau – Kölntourismus Gmbh
3                         Convention Bureau Düsseldorf
4                      Convention Bureau Lower Austria
5                      Convention Bureau Upper Austria
6                                 Convention Edinburgh
7                      Croatian National Tourist Board
8    Cyprus Tourism Organisation - Cyprus Conventio...
9               Czechtourism - Czech Convention Bureau
10                       La Cité des Congrès de Nantes
11                            Dublin Convention Bureau
12                         Edinburgh Conference Centre
13                           Finland Convention Bureau
14                         Frankfurt Convention Bureau
15            Georgia Convention And Exhibition Bureau
16                      German Convention Bureau (Gcb)
17        

In [5]:
fdata['Company'] = fdata['Company'].str.replace('[{}]'.format(string.punctuation), '')
fdata['Company'] = fdata['Company'].str.replace('-', '')
fdata['Company'] = fdata['Company'].str.replace('.', '')
fdata['Company'] = fdata['Company'].str.replace(' - ', '')

In [6]:
fdata

,Company
0,Barcelona Convention Bureau
1,Belgium VisitBrussels Convention Bureau
2,Cologne Convention Bureau – Kölntourismus Gmbh
3,Convention Bureau Düsseldorf
4,Convention Bureau Lower Austria
5,Convention Bureau Upper Austria
6,Convention Edinburgh
7,Croatian National Tourist Board
8,Cyprus Tourism Organisation Cyprus Convention...
9,Czechtourism Czech Convention Bureau


In [7]:
# Create sequence
vocabulary_size = 3500
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(fdata['Company'])
sequences = tokenizer.texts_to_sequences(fdata['Company'])
data = pad_sequences(sequences, maxlen=6, padding = 'post', truncating = 'pre')

In [8]:
data[0]

array([41,  2,  3,  0,  0,  0])

In [9]:
model = Sequential()
model.add(Embedding(3500, 100, input_length=6))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.4))
model.add(Dense(11, activation='relu'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [10]:
ldata['Category'] = ldata['Category'].str.replace('Congress center', 'CC')
ldata['Category'] = ldata['Category'].str.replace('Conference center', 'CC')
ldata.loc[ldata.Category == 'T', 'Category'] = 'TA'
ldata

,Category
0,CB
1,CB
2,CB
3,CB
4,CB
5,CB
6,CB
7,TB
8,CB
9,CB


In [11]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
y=ldata["Category"]
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(y)
print(integer_encoded)
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)

[ 1  1  1  1  1  1  1  8  1  1  2  1  2  1  1  1  1  1  1  1  1  9  1  1
  9  8  1  1  1  1  1  1  1  2  1  2  1  1  1  1  1  1  9  9  9  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  9  7  7  7  7  7  5
  5  5  5  3  5  6  5  5  3  3  3  5  5  3  3  5  5  5  5  5  5  5  3  3
 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10  4  7
  7  7  7  7  7  7  7  7 10 10  7  4 10 10 10 10 10  6  6  6  6 10  5  5
  6  4 10 10 10 10 10 10  4 10 10 10 10 10  4 10 10  4 10 10  3  3  3  3
  3  6  6  3  3  5  3  5  6  5  5  5  5  5  3  3  5  5  6  5  5  5  5  5
  3  5  5  5  5  5 10 10 10 10 10 10  4  4 10  9  9  9  9  9  9  9  9  9
  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9
  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9
  9  9  9 10 10 10 10  4 10 10 10 10 10  4 10 10  4 10 10 10 10  6  5  5
  3  1  5  8  4 10 10  4 10 10  7 10  9  5 10 10  5  5  3  7 10 10 10  5
  4  5 10 10  3  4  4 10 10 10  5  5  5  3  5  1  7

In [12]:
ldata.Category.unique()

array(['CB', 'TB', 'CC', 'TO', 'TA', 'H', 'CH', 'R', 'WP', 'EP', 'A'],
      dtype=object)

In [13]:
model.fit(data, np.array(onehot_encoded), validation_split=0.4, epochs=1000)

Train on 306 samples, validate on 205 samples
Epoch 1/1000
306/306 [==============================] - 2s 8ms/step - loss: 1.1269 - acc: 0.9091 - val_loss: 0.6963 - val_acc: 0.9091
Epoch 2/1000
306/306 [==============================] - 0s 828us/step - loss: 0.7087 - acc: 0.9091 - val_loss: 0.6059 - val_acc: 0.9091
Epoch 3/1000
306/306 [==============================] - 0s 845us/step - loss: 0.6355 - acc: 0.9091 - val_loss: 0.6262 - val_acc: 0.9091
Epoch 4/1000
306/306 [==============================] - 0s 937us/step - loss: 0.6257 - acc: 0.9091 - val_loss: 0.6960 - val_acc: 0.9091
Epoch 5/1000
306/306 [==============================] - 0s 948us/step - loss: 0.6231 - acc: 0.9091 - val_loss: 0.7886 - val_acc: 0.9091
Epoch 6/1000
306/306 [==============================] - 0s 829us/step - loss: 0.6129 - acc: 0.9091 - val_loss: 0.6793 - val_acc: 0.9091
Epoch 7/1000
306/306 [==============================] - 0s 932us/step - loss: 0.6098 - acc: 0.9091 - val_loss: 0.5844 - val_acc: 0.9091
Epoc

Epoch 120/1000
306/306 [==============================] - 0s 830us/step - loss: 0.0021 - acc: 0.9207 - val_loss: 0.8633 - val_acc: 0.9002
Epoch 121/1000
306/306 [==============================] - 0s 808us/step - loss: 0.0021 - acc: 0.9204 - val_loss: 0.8627 - val_acc: 0.8993
Epoch 122/1000
306/306 [==============================] - 0s 830us/step - loss: 0.0018 - acc: 0.9177 - val_loss: 0.8624 - val_acc: 0.8993
Epoch 123/1000
306/306 [==============================] - 0s 807us/step - loss: 0.0028 - acc: 0.9201 - val_loss: 0.8632 - val_acc: 0.8993
Epoch 124/1000
306/306 [==============================] - 0s 830us/step - loss: 0.0029 - acc: 0.9174 - val_loss: 0.8491 - val_acc: 0.8998
Epoch 125/1000
306/306 [==============================] - 0s 859us/step - loss: 0.0022 - acc: 0.9201 - val_loss: 0.8419 - val_acc: 0.9016
Epoch 126/1000
306/306 [==============================] - 0s 830us/step - loss: 0.0028 - acc: 0.9198 - val_loss: 0.8299 - val_acc: 0.9020
Epoch 127/1000
306/306 [==========

Epoch 236/1000
306/306 [==============================] - 0s 830us/step - loss: 1.9006e-04 - acc: 0.9124 - val_loss: 1.0448 - val_acc: 0.8976
Epoch 237/1000
306/306 [==============================] - 0s 858us/step - loss: 4.9223e-04 - acc: 0.9141 - val_loss: 1.0454 - val_acc: 0.8976
Epoch 238/1000
306/306 [==============================] - 0s 830us/step - loss: 6.2826e-04 - acc: 0.9130 - val_loss: 1.0243 - val_acc: 0.8971
Epoch 239/1000
306/306 [==============================] - 0s 842us/step - loss: 4.7054e-04 - acc: 0.9135 - val_loss: 1.0101 - val_acc: 0.8984
Epoch 240/1000
306/306 [==============================] - 0s 846us/step - loss: 3.7648e-06 - acc: 0.9121 - val_loss: 0.9962 - val_acc: 0.8984
Epoch 241/1000
306/306 [==============================] - 0s 830us/step - loss: 1.2633e-04 - acc: 0.9124 - val_loss: 0.9814 - val_acc: 0.8984
Epoch 242/1000
306/306 [==============================] - 0s 859us/step - loss: 1.9505e-04 - acc: 0.9130 - val_loss: 0.9782 - val_acc: 0.8984
Epoch 

Epoch 351/1000
306/306 [==============================] - 0s 830us/step - loss: 4.4141e-06 - acc: 0.9115 - val_loss: 0.9991 - val_acc: 0.8927
Epoch 352/1000
306/306 [==============================] - 0s 830us/step - loss: 1.0175e-07 - acc: 0.9124 - val_loss: 0.9991 - val_acc: 0.8927
Epoch 353/1000
306/306 [==============================] - 0s 808us/step - loss: 1.8219e-04 - acc: 0.9115 - val_loss: 1.0116 - val_acc: 0.8922
Epoch 354/1000
306/306 [==============================] - 0s 829us/step - loss: 3.8974e-05 - acc: 0.9100 - val_loss: 1.0027 - val_acc: 0.8918
Epoch 355/1000
306/306 [==============================] - 0s 860us/step - loss: 1.7450e-05 - acc: 0.9106 - val_loss: 1.0044 - val_acc: 0.8918
Epoch 356/1000
306/306 [==============================] - 0s 830us/step - loss: 1.1886e-05 - acc: 0.9106 - val_loss: 1.0040 - val_acc: 0.8914
Epoch 357/1000
306/306 [==============================] - 0s 859us/step - loss: 3.4908e-05 - acc: 0.9103 - val_loss: 1.0026 - val_acc: 0.8922
Epoch 

Epoch 466/1000
306/306 [==============================] - 0s 881us/step - loss: 1.0175e-07 - acc: 0.9100 - val_loss: 1.0701 - val_acc: 0.8914
Epoch 467/1000
306/306 [==============================] - 0s 829us/step - loss: 1.0175e-07 - acc: 0.9106 - val_loss: 1.0701 - val_acc: 0.8914
Epoch 468/1000
306/306 [==============================] - 0s 808us/step - loss: 1.0175e-07 - acc: 0.9100 - val_loss: 1.0701 - val_acc: 0.8914
Epoch 469/1000
306/306 [==============================] - 0s 881us/step - loss: 1.0175e-07 - acc: 0.9100 - val_loss: 1.0701 - val_acc: 0.8914
Epoch 470/1000
306/306 [==============================] - 0s 808us/step - loss: 3.4044e-05 - acc: 0.9103 - val_loss: 1.0699 - val_acc: 0.8914
Epoch 471/1000
306/306 [==============================] - 0s 829us/step - loss: 7.7662e-05 - acc: 0.9112 - val_loss: 1.0698 - val_acc: 0.8918
Epoch 472/1000
306/306 [==============================] - 0s 859us/step - loss: 1.0175e-07 - acc: 0.9109 - val_loss: 1.0698 - val_acc: 0.8918
Epoch 

...

Epoch 698/1000
306/306 [==============================] - 0s 860us/step - loss: 1.0175e-07 - acc: 0.9091 - val_loss: 1.1165 - val_acc: 0.8909
Epoch 699/1000
306/306 [==============================] - 0s 829us/step - loss: 1.1021e-04 - acc: 0.9100 - val_loss: 1.1031 - val_acc: 0.8891
Epoch 700/1000
306/306 [==============================] - 0s 859us/step - loss: 1.0175e-07 - acc: 0.9094 - val_loss: 1.0829 - val_acc: 0.8896
Epoch 701/1000
306/306 [==============================] - 0s 779us/step - loss: 1.0175e-07 - acc: 0.9094 - val_loss: 1.0864 - val_acc: 0.8891
Epoch 702/1000
306/306 [==============================] - 0s 808us/step - loss: 1.0175e-07 - acc: 0.9097 - val_loss: 1.0861 - val_acc: 0.8882
Epoch 703/1000
306/306 [==============================] - 0s 880us/step - loss: 3.7107e-04 - acc: 0.9100 - val_loss: 0.9865 - val_acc: 0.8936
Epoch 704/1000
306/306 [==============================] - 0s 830us/step - loss: 1.0175e-07 - acc: 0.9103 - val_loss: 0.9624 - val_acc: 0.8980
Epoch 

Epoch 813/1000
306/306 [==============================] - 0s 859us/step - loss: 1.0175e-07 - acc: 0.9091 - val_loss: 1.0781 - val_acc: 0.8900
Epoch 814/1000
306/306 [==============================] - 0s 830us/step - loss: 5.8524e-05 - acc: 0.9091 - val_loss: 1.0827 - val_acc: 0.8900
Epoch 815/1000
306/306 [==============================] - 0s 808us/step - loss: 1.0175e-07 - acc: 0.9100 - val_loss: 1.0880 - val_acc: 0.8909
Epoch 816/1000
306/306 [==============================] - 0s 829us/step - loss: 1.0175e-07 - acc: 0.9097 - val_loss: 1.0882 - val_acc: 0.8914
Epoch 817/1000
306/306 [==============================] - 0s 861us/step - loss: 1.0175e-07 - acc: 0.9094 - val_loss: 1.0884 - val_acc: 0.8914
Epoch 818/1000
306/306 [==============================] - 0s 829us/step - loss: 1.0175e-07 - acc: 0.9094 - val_loss: 1.0885 - val_acc: 0.8914
Epoch 819/1000
306/306 [==============================] - 0s 830us/step - loss: 1.0175e-07 - acc: 0.9097 - val_loss: 1.0885 - val_acc: 0.8914
Epoch 

Epoch 871/1000
306/306 [==============================] - 0s 830us/step - loss: 1.0175e-07 - acc: 0.9103 - val_loss: 1.0615 - val_acc: 0.8927
Epoch 872/1000
306/306 [==============================] - 0s 860us/step - loss: 1.0175e-07 - acc: 0.9100 - val_loss: 1.0612 - val_acc: 0.8927
Epoch 873/1000
306/306 [==============================] - 0s 830us/step - loss: 1.0175e-07 - acc: 0.9103 - val_loss: 1.0611 - val_acc: 0.8931
Epoch 874/1000
306/306 [==============================] - 0s 809us/step - loss: 1.0175e-07 - acc: 0.9103 - val_loss: 1.0611 - val_acc: 0.8931
Epoch 875/1000
306/306 [==============================] - 0s 880us/step - loss: 1.8921e-05 - acc: 0.9112 - val_loss: 1.0671 - val_acc: 0.8927
Epoch 876/1000
306/306 [==============================] - 0s 830us/step - loss: 1.0175e-07 - acc: 0.9109 - val_loss: 1.0700 - val_acc: 0.8922
Epoch 877/1000
306/306 [==============================] - 0s 860us/step - loss: 1.0175e-07 - acc: 0.9109 - val_loss: 1.0693 - val_acc: 0.8922
Epoch 

Epoch 929/1000
306/306 [==============================] - 0s 809us/step - loss: 3.2311e-04 - acc: 0.9091 - val_loss: 1.1166 - val_acc: 0.8891
Epoch 930/1000
306/306 [==============================] - 0s 830us/step - loss: 1.5773e-05 - acc: 0.9097 - val_loss: 1.1057 - val_acc: 0.8896
Epoch 931/1000
306/306 [==============================] - 0s 808us/step - loss: 1.0160e-06 - acc: 0.9097 - val_loss: 1.1055 - val_acc: 0.8891
Epoch 932/1000
306/306 [==============================] - 0s 830us/step - loss: 1.0175e-07 - acc: 0.9094 - val_loss: 1.1008 - val_acc: 0.8896
Epoch 933/1000
306/306 [==============================] - 0s 881us/step - loss: 2.1166e-06 - acc: 0.9097 - val_loss: 1.1052 - val_acc: 0.8896
Epoch 934/1000
306/306 [==============================] - 0s 808us/step - loss: 1.0175e-07 - acc: 0.9094 - val_loss: 1.1054 - val_acc: 0.8896
Epoch 935/1000
306/306 [==============================] - 0s 830us/step - loss: 1.0175e-07 - acc: 0.9094 - val_loss: 1.1055 - val_acc: 0.8896
Epoch 

Epoch 987/1000
306/306 [==============================] - 0s 881us/step - loss: 1.0175e-07 - acc: 0.9094 - val_loss: 0.9958 - val_acc: 0.8936
Epoch 988/1000
306/306 [==============================] - 0s 816us/step - loss: 4.8908e-05 - acc: 0.9112 - val_loss: 1.0340 - val_acc: 0.8905
Epoch 989/1000
306/306 [==============================] - 0s 873us/step - loss: 1.0175e-07 - acc: 0.9103 - val_loss: 1.0399 - val_acc: 0.8882
Epoch 990/1000
306/306 [==============================] - 0s 830us/step - loss: 6.9655e-05 - acc: 0.9091 - val_loss: 1.0460 - val_acc: 0.8878
Epoch 991/1000
306/306 [==============================] - 0s 860us/step - loss: 1.0477e-05 - acc: 0.9100 - val_loss: 1.0307 - val_acc: 0.8878
Epoch 992/1000
306/306 [==============================] - 0s 829us/step - loss: 1.0175e-07 - acc: 0.9100 - val_loss: 1.0274 - val_acc: 0.8882
Epoch 993/1000
306/306 [==============================] - 0s 860us/step - loss: 1.0175e-07 - acc: 0.9094 - val_loss: 1.0318 - val_acc: 0.8891
Epoch 

In [14]:
# model.fit(data, np.array(onehot_encoded), validation_split=0.4, epochs=100)

In [15]:
model.save_weights('OTM_weights.h5')

In [16]:
nocat = pd.read_excel("NoCat.xlsx")
nocat

,Category,Status,Company,Contact Person,Designation,Decision maker,Email ID,Phone,Notes prospects (for sentiment analisys),Country,Website,Address,Notes sales process,Unnamed: 13,Unnamed: 14,Unnamed: 15
577,NaN,Cold lead,CRB CITYRAMA Travel Agency,Tünde MOGYORÓSI,Manager of Incoming Department,yes,Cityrama@cityrama.hu,NaN,NaN,Hungary,NaN,NaN,"If I have to hit a button only, I would hit: 4...",NaN,NaN,NaN
588,NaN,NaN,HOTEL INTERNATIONAL IASI,AlexandruHOLBANEL,Sales Executive,yes,sales@hotelinternationaliasi.ro,NaN,NaN,Romania,NaN,NaN,NaN,NaN,NaN,NaN
589,NaN,NaN,Tirolwerbung,AngelaSemrajc,Employee,yes,Angela.semrajc@tirolwerbung.at,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
591,NaN,NaN,Polish National Tourist Office,AnnaCichońska,Director,yes,NaN,NaN,USA and Canda market,Poland,NaN,NaN,NaN,NaN,NaN,NaN
592,NaN,NaN,Rynek Turystyczny,AnnaJanuszewska,Marketing and Commercials Specialist,yes,a.januszewska@pws-promedia.pl,NaN,This is Tourist Magazine - partner with Polski...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
593,NaN,NaN,Zadar Tourist Board,AnteGalić,Public Relations & Marketing,yes,ante@zadar.travel,NaN,NaN,Croatia,NaN,NaN,NaN,NaN,NaN,NaN
594,NaN,NaN,Biuro Wadi,ArturPiszczek,Owner,yes,biuro@wadi.pl,NaN,NaN,Poland,NaN,NaN,NaN,NaN,NaN,NaN
595,NaN,NaN,Emilia Kubik,BarbaraTutak,"Główny Specjalista, Lider Projektu",yes,barbara.tutak@pot.gov.pl,NaN,NaN,Poland,NaN,NaN,NaN,NaN,NaN,NaN
596,NaN,NaN,Orthanna Spltd,BoyanaKostadinova,Manager,yes,b.kostadinova@orthanna.com,NaN,NaN,Bulgaria,NaN,NaN,NaN,NaN,NaN,NaN
597,NaN,NaN,88 Rooms,BranimirMladenovic,Sales and marketing administrator,yes,branimir.mladenovic@88rooms.com,NaN,NaN,Serbia,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
nc = nocat.loc[0:899,['Company']]

In [18]:
nc

,Company
577,CRB CITYRAMA Travel Agency
588,HOTEL INTERNATIONAL IASI
589,Tirolwerbung
591,Polish National Tourist Office
592,Rynek Turystyczny
593,Zadar Tourist Board
594,Biuro Wadi
595,Emilia Kubik
596,Orthanna Spltd
597,88 Rooms


In [19]:
nc['Company'] = nc['Company'].str.replace('[{}]'.format(string.punctuation), '')
nc['Company'] = nc['Company'].str.replace('-', '')
nc['Company'] = nc['Company'].str.replace('.', '')
nc['Company'] = nc['Company'].str.replace(' - ', '')
nc

,Company
577,CRB CITYRAMA Travel Agency
588,HOTEL INTERNATIONAL IASI
589,Tirolwerbung
591,Polish National Tourist Office
592,Rynek Turystyczny
593,Zadar Tourist Board
594,Biuro Wadi
595,Emilia Kubik
596,Orthanna Spltd
597,88 Rooms


In [20]:
### Create sequence
vocabulary_size = 3500
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(nc['Company'])
sequences = tokenizer.texts_to_sequences(nc['Company'])
d = pad_sequences(sequences, maxlen=6, padding = 'post', truncating = 'pre')

In [21]:
model.predict(d, verbose = 1)

112/112 [==============================] - 0s 1ms/step


array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.6972561 , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.27903023, 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 1.2438273 , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.812913  , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]], dtype=float32)

In [22]:
prediction = model.predict(d)

In [23]:
prediction

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.6972561 , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.27903023, 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 1.2438273 , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.812913  , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]], dtype=float32)

In [24]:
prediction[1]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.02376377, 0.        , 0.        , 0.6972561 , 0.        ,
       0.        ], dtype=float32)

In [25]:
p =(prediction == prediction.max(axis=1)[:,None]).astype(float)

In [26]:
p[1]

array([0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.])

In [27]:
df = pd.DataFrame(p)
df

,0,1,2,3,4,5,6,7,8,9,10
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
8,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [28]:
df['Category'] = 'x'
df

,0,1,2,3,4,5,6,7,8,9,10,Category
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,x
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,x
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,x
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,x
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,x
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,x
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,x
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,x
8,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,x
9,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,x


In [29]:
df.loc[df[0] == 1.0, 'Category'] = 'A'
df.loc[df[1] == 1.0, 'Category'] = 'CB'
df.loc[df[2] == 1.0, 'Category'] = 'CC'
df.loc[df[3] == 1.0, 'Category'] = 'CH'
df.loc[df[4] == 1.0, 'Category'] = 'EP'
df.loc[df[5] == 1.0, 'Category'] = 'H'
df.loc[df[6] == 1.0, 'Category'] = 'R'
df.loc[df[7] == 1.0, 'Category'] = 'TA'
df.loc[df[8] == 1.0, 'Category'] = 'TB'
df.loc[df[9] == 1.0, 'Category'] = 'TO'
df.loc[df[10] == 1.0, 'Category'] = 'WP'

In [30]:
df

,0,1,2,3,4,5,6,7,8,9,10,Category
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,R
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,TB
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,TB
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,TO
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,TO
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,WP
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,TB
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,TB
8,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CB
9,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,H


In [31]:
x = nocat['Company']
x.index = range(112)
x = pd.DataFrame(x)
x

,Company
0,CRB CITYRAMA Travel Agency
1,HOTEL INTERNATIONAL IASI
2,Tirolwerbung
3,Polish National Tourist Office
4,Rynek Turystyczny
5,Zadar Tourist Board
6,Biuro Wadi
7,Emilia Kubik
8,Orthanna Spltd
9,88 Rooms


In [32]:
df = pd.concat([df, x['Company']], axis = 1)

In [33]:
df = df[['Category', 'Company']]
df

,Category,Company
0,R,CRB CITYRAMA Travel Agency
1,TB,HOTEL INTERNATIONAL IASI
2,TB,Tirolwerbung
3,TO,Polish National Tourist Office
4,TO,Rynek Turystyczny
5,WP,Zadar Tourist Board
6,TB,Biuro Wadi
7,TB,Emilia Kubik
8,CB,Orthanna Spltd
9,H,88 Rooms


In [35]:
df.to_excel("pred7.xlsx")

In [36]:
df['Category'].value_counts()

TB    21
H     20
WP    19
TO    16
CB    14
CH     8
EP     5
R      4
CC     3
TA     2
Name: Category, dtype: int64